#### Creates an HTML Cluster Map for Data from AFDC API


In [ ]:
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
import pandas as pd

############################## local data 
df = pd.read_csv('fuel_data.csv')
#unique values for first column
df['Fuel Type Code'].unique()
df.columns

# Define a list of columns relevant to electric fuel types
df = pd.read_csv('stations_new.csv')
relevant_columns = [
    'Station Name', 'Street Address', 'City', 'State', 'ZIP', 'Latitude', 'Longitude',
    'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Network',
    'EV Network Web', 'Access Days Time', 'Station Phone', 'EV Connector Types'
]

# Drop columns that are not in the list of relevant columns
df_elec = df[relevant_columns].copy()
# Assuming df_elec is your DataFrame filtered for electric fuel types
df = pd.read_csv('fuel_data.csv')

df_elec = df[df['Fuel Type Code'] == 'ELEC'].copy()
# Filter the DataFrame for stations in California
df_elec_ca = df_elec[df_elec['State'] == 'CA'].copy()

# Initialize the map centered around California
m_elec_ca = folium.Map(location=[36.7783, -119.4179], zoom_start=6)

# Create a marker cluster for the entire map
mc_elec_ca = MarkerCluster(name="All Stations", show=True)
m_elec_ca.add_child(mc_elec_ca)

# Loop through the filtered DataFrame to add each EV station to the cluster
for idx, row in df_elec_ca.iterrows():
    popup_text = f"""
    <strong>Station Name:</strong> {row['Station Name']}<br>
    <strong>Address:</strong> {row['Street Address']}, {row['City']}, {row['State']} {row['ZIP']}<br>
    <strong>Phone:</strong> {row['Station Phone']}<br>
    <strong>EV Level 1 EVSE:</strong> {row['EV Level1 EVSE Num']}<br>
    <strong>EV Level 2 EVSE:</strong> {row['EV Level2 EVSE Num']}<br>
    <strong>EV DC Fast Count:</strong> {row['EV DC Fast Count']}<br>
    <strong>EV Network:</strong> {row['EV Network']}
    """
    mc_elec_ca.add_child(folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_text, max_width=300)
    ))


# Add GeoJSON layers from ZIP files
colors = ['green', 'red', 'blue']
shapefile_paths = [
    'Bottlenecks/HWY_Bottleneck.shp',
    'Traffic_Volumes_AADT/HWY_Traffic_Volumes_AADT.shp',
    'Average_Transit_Speeds_by_Route_Time-of-Day_and_Segments/Average_Transit_Speeds_by_Route_Time-of-Day_and_Segments.shp']


for shp_path, color in zip(shapefile_paths, colors):
    gdf = gpd.read_file(shp_path)
    style_function = lambda x: {'color': color, 'weight': 3 if 'LineString' in x['geometry'].type else 1}
    folium.GeoJson(
        gdf,
        style_function=style_function,
        name=shp_path.split('/')[-1].split('.')[0]  # Naming by shapefile name
    ).add_to(m_elec_ca)

folium.LayerControl().add_to(m_elec_ca)

##print map 
m_elec_ca
# Save the map
#m_elec_ca.save('ev_ca_map.html')
